In [1]:
import pandas as pd
import warnings
import os
import sys
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import missingno as msno

root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

sys.path.insert(0, root_dir)

from config import DATA_PATH

# Show all columns
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv(os.path.join(DATA_PATH, 'expanded_data.csv'))

df.head()

,day_of_week,age_band_of_driver,educational_level,driving_experience,service_year_of_vehicle,defect_of_vehicle,number_of_vehicles_involved,number_of_casualties,age_band_of_casualty,casualty_severity,accident_severity,sex_of_driver_female,sex_of_driver_male,type_of_vehicle_automobile,type_of_vehicle_bajaj,type_of_vehicle_bicycle,type_of_vehicle_long lorry,type_of_vehicle_lorry (11?40q),type_of_vehicle_lorry (41?100q),type_of_vehicle_motorcycle,type_of_vehicle_other,type_of_vehicle_pick up upto 10q,type_of_vehicle_public (12 seats),type_of_vehicle_public (13?45 seats),type_of_vehicle_public (> 45 seats),type_of_vehicle_ridden horse,type_of_vehicle_special vehicle,type_of_vehicle_stationwagen,type_of_vehicle_taxi,type_of_vehicle_turbo,owner_of_vehicle_governmental,owner_of_vehicle_organization,owner_of_vehicle_other,owner_of_vehicle_owner,area_accident_occured_church areas,area_accident_occured_hospital areas,area_accident_occured_industrial areas,area_accident_occured_market areas,area_accident_occured_office areas,area_accident_occured_other,area_accident_occured_outside rural areas,area_accident_occured_recreational areas,area_accident_occured_residential areas,area_accident_occured_rural village areas,area_accident_occured_school areas,lanes_or_medians_divided two-way,lanes_or_medians_double carriageway (median),lanes_or_medians_one way,lanes_or_medians_other,lanes_or_medians_undivided two way,road_allignment_escarpments,road_allignment_gentle horizontal curve,road_allignment_sharp reverse curve,road_allignment_steep grade downward with mountainous terrain,road_allignment_steep grade upward with mountainous terrain,road_allignment_tangent road with flat terrain,road_allignment_tangent road with mild grade and flat terrain,road_allignment_tangent road with mountainous terrain and,road_allignment_tangent road with rolling terrain,types_of_junction_crossing,types_of_junction_no junction,types_of_junction_o shape,types_of_junction_other,types_of_junction_t shape,types_of_junction_x shape,types_of_junction_y shape,road_surface_type_aspahlt roads,road_surface_type_asphalt roads,road_surface_type_earth roads,road_surface_type_gravel roads,road_surface_type_other,road_surface_conditions_dry,road_surface_conditions_flood over 3cm. deep,road_surface_conditions_snow,road_surface_conditions_wet or damp,light_conditions_darkness,light_conditions_daylight,weather_conditions_cloudy,weather_conditions_fog or mist,weather_conditions_normal,weather_conditions_other,weather_conditions_raining,weather_conditions_raining and windy,weather_conditions_snow,weather_conditions_windy,type_of_collision_collision with animals,type_of_collision_collision with pedestrians,type_of_collision_collision with roadside objects,type_of_collision_collision with roadside-parked vehicles,type_of_collision_fall from vehicles,type_of_collision_other,type_of_collision_rollover,type_of_collision_vehicle with vehicle collision,type_of_collision_with train,vehicle_movement_entering a junction,vehicle_movement_getting off,vehicle_movement_going straight,vehicle_movement_moving backward,vehicle_movement_other,vehicle_movement_overtaking,vehicle_movement_parked,vehicle_movement_reversing,vehicle_movement_stopping,vehicle_movement_turnover,vehicle_movement_u-turn,vehicle_movement_waiting to go,casualty_class_driver or rider,casualty_class_passenger,casualty_class_pedestrian,sex_of_casualty_female,sex_of_casualty_male,work_of_casuality_driver,work_of_casuality_employee,work_of_casuality_other,work_of_casuality_self-employed,work_of_casuality_student,work_of_casuality_unemployed,fitness_of_casuality_blind,fitness_of_casuality_deaf,fitness_of_casuality_normal,fitness_of_casuality_other,pedestrian_movement_crossing from driver's nearside,pedestrian_movement_crossing from nearside - masked by parked or stationot a pedestrianry vehicle,pedestrian_movement_crossing from offside - masked by parked or stationot a pedestrianry vehicle,"pedestrian_movement_in carriageway, stationot a p

In [8]:
# Store columns with missing values in a list
missing_cols = df.columns[df.isnull().any()].tolist()

In [12]:
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")
clean_data = df.copy()
clean_data[missing_cols] = knn_imputer.fit_transform(clean_data[missing_cols])

In [13]:
clean_data.head()

,day_of_week,age_band_of_driver,educational_level,driving_experience,service_year_of_vehicle,defect_of_vehicle,number_of_vehicles_involved,number_of_casualties,age_band_of_casualty,casualty_severity,accident_severity,sex_of_driver_female,sex_of_driver_male,type_of_vehicle_automobile,type_of_vehicle_bajaj,type_of_vehicle_bicycle,type_of_vehicle_long lorry,type_of_vehicle_lorry (11?40q),type_of_vehicle_lorry (41?100q),type_of_vehicle_motorcycle,type_of_vehicle_other,type_of_vehicle_pick up upto 10q,type_of_vehicle_public (12 seats),type_of_vehicle_public (13?45 seats),type_of_vehicle_public (> 45 seats),type_of_vehicle_ridden horse,type_of_vehicle_special vehicle,type_of_vehicle_stationwagen,type_of_vehicle_taxi,type_of_vehicle_turbo,owner_of_vehicle_governmental,owner_of_vehicle_organization,owner_of_vehicle_other,owner_of_vehicle_owner,area_accident_occured_church areas,area_accident_occured_hospital areas,area_accident_occured_industrial areas,area_accident_occured_market areas,area_accident_occured_office areas,area_accident_occured_other,area_accident_occured_outside rural areas,area_accident_occured_recreational areas,area_accident_occured_residential areas,area_accident_occured_rural village areas,area_accident_occured_school areas,lanes_or_medians_divided two-way,lanes_or_medians_double carriageway (median),lanes_or_medians_one way,lanes_or_medians_other,lanes_or_medians_undivided two way,road_allignment_escarpments,road_allignment_gentle horizontal curve,road_allignment_sharp reverse curve,road_allignment_steep grade downward with mountainous terrain,road_allignment_steep grade upward with mountainous terrain,road_allignment_tangent road with flat terrain,road_allignment_tangent road with mild grade and flat terrain,road_allignment_tangent road with mountainous terrain and,road_allignment_tangent road with rolling terrain,types_of_junction_crossing,types_of_junction_no junction,types_of_junction_o shape,types_of_junction_other,types_of_junction_t shape,types_of_junction_x shape,types_of_junction_y shape,road_surface_type_aspahlt roads,road_surface_type_asphalt roads,road_surface_type_earth roads,road_surface_type_gravel roads,road_surface_type_other,road_surface_conditions_dry,road_surface_conditions_flood over 3cm. deep,road_surface_conditions_snow,road_surface_conditions_wet or damp,light_conditions_darkness,light_conditions_daylight,weather_conditions_cloudy,weather_conditions_fog or mist,weather_conditions_normal,weather_conditions_other,weather_conditions_raining,weather_conditions_raining and windy,weather_conditions_snow,weather_conditions_windy,type_of_collision_collision with animals,type_of_collision_collision with pedestrians,type_of_collision_collision with roadside objects,type_of_collision_collision with roadside-parked vehicles,type_of_collision_fall from vehicles,type_of_collision_other,type_of_collision_rollover,type_of_collision_vehicle with vehicle collision,type_of_collision_with train,vehicle_movement_entering a junction,vehicle_movement_getting off,vehicle_movement_going straight,vehicle_movement_moving backward,vehicle_movement_other,vehicle_movement_overtaking,vehicle_movement_parked,vehicle_movement_reversing,vehicle_movement_stopping,vehicle_movement_turnover,vehicle_movement_u-turn,vehicle_movement_waiting to go,casualty_class_driver or rider,casualty_class_passenger,casualty_class_pedestrian,sex_of_casualty_female,sex_of_casualty_male,work_of_casuality_driver,work_of_casuality_employee,work_of_casuality_other,work_of_casuality_self-employed,work_of_casuality_student,work_of_casuality_unemployed,fitness_of_casuality_blind,fitness_of_casuality_deaf,fitness_of_casuality_normal,fitness_of_casuality_other,pedestrian_movement_crossing from driver's nearside,pedestrian_movement_crossing from nearside - masked by parked or stationot a pedestrianry vehicle,pedestrian_movement_crossing from offside - masked by parked or stationot a pedestrianry vehicle,"pedestrian_movement_in carriageway, stationot a p

In [15]:
selector = SelectKBest(chi2, k=10)
selector.fit(clean_data, df['accident_severity'])

top_features_indices = selector.get_support(indices=True)

In [16]:
top_features_indices

array([ 0,  5,  6,  7, 10, 59, 60, 75, 86, 99])

In [17]:
data_selected = clean_data.iloc[:, top_features_indices]

In [18]:
data_selected.head()

,day_of_week,defect_of_vehicle,number_of_vehicles_involved,number_of_casualties,accident_severity,types_of_junction_crossing,types_of_junction_no junction,light_conditions_darkness,type_of_collision_collision with pedestrians,vehicle_movement_overtaking
0,1,0.0,2,2,1,0,1,0,0,0
1,1,0.0,2,2,1,0,1,0,0,0
2,1,0.0,2,2,2,0,1,0,0,0
3,7,0.0,2,2,1,0,0,1,0,0
4,7,0.0,2,2,1,0,0,1,0,0


In [19]:
clean_data.to_csv(os.path.join(DATA_PATH, 'clean_data.csv'), index=False)